# Distance Notebook...
Using to test this, I have no direction

### Imports and Setup

In [ ]:
import os
import ast
import numpy as np
import pandas as pd
from typing import Any
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
os.chdir('/Users/nico/Desktop/CIIC/CAPSTONE/essentia_demo/')

from src.utils.distance import *
# NOTE : You have to change the working directory to the root (essentia_demo) directory
#        for the utils import to work. You can use `os.chdir(path)` for this.


DATASET_PATH   = '/Users/nico/Desktop/CIIC/CAPSTONE/essentia_demo/datasets/full_dataset_w_harmo.csv'
track_df       = pd.read_csv(DATASET_PATH)

numerical_fts  = ['popularity', 'mfcc_peak_energy', 'mfcc_avg_energy'	,'pitch_salience', 'avg_dissonance', 'avg_rollof', 'bpm', 'energy', 'integrated_loudness',
                  'loudness_range',	'danceable_effnet',	'aggressive_effnet', 'happy_effnet', 'party_effnet', 'relaxed_effnet', 'sad_effnet', 'acoustic_effnet',
                  'electronic_effnet',	'instrumental_effnet', 'female_effnet',	'tonal_effnet',	'bright_effnet','bright_nsynth_effnet',	'approachable_effnet',
                  'approachability_effnet',	'engaging_effnet',	'engagement_effnet', 'pitch_mean', 'pitch_var']

# numerical_fts   = ['popularity']
# dimensional_fts = ['most_common_chords', 'most_common_keys', 'tristimulus', 'pitch_hist']


# Before doing anything, I'll first normalize the dataset using Z-Score Normalization which is what best worked for me previously. I'll also do some other shit man.
# deepcopy the original dataframe so that it remains untouched.
z_score_df                = deepcopy(track_df)
z_score_df[numerical_fts] = StandardScaler().fit_transform(z_score_df[numerical_fts])

### General Helpers
Quickly define some helper functions to do all the distance comparisons and whatnot.

In [ ]:
def dist_helper(input_track : pd.Series, compare_df : pd.DataFrame,
                num_feats : dict[str], dim_feats : dict[str] = None, top_ix : int = 15):
    """Helps out w/ getting the distance score for a given `input_track` against the `compare_df`. It finalizes
    by printing the `top_ix` tracks with the closest distance scores.

    Args:
        input_track  : The track which we will use to compare against the `compare_df` and which we will try to provide
                       accurate recommendations for. This track will be a single row taken from the `compare_df` at the
                       moment since we're currently testing.
        compare_df   : The dataframe containing all of our tracks which we will use as our recommendation pool.
        num_feats    : The set of numerical features which will be used to evaluate and compare tracks. Contains weights as their values.
        dim_feats    : The set of dimensional features (e.g. tristimulus = (X,Y,Z)) used to evaluate and compare tracks.
        top_ix       : The top number of closest tracks to be displayed.
    """
    result_dict = {}
    for _, curr_track in compare_df.iterrows():
        track_name = "-".join([curr_track['artist'], curr_track['title']])
        dist_score = euclidean_numerical(num_feats, input_track, curr_track)          # Euclidean Dist using minkowski (p=2)
        if dim_feats:                                                                 # NOTE : Needs changing, but it's ok, just testing rn!
            dist_score += euclidean_dimensional(dim_feats, input_track, curr_track)
        result_dict[track_name] = dist_score


    output = dict(sorted(result_dict.items(), key=lambda item: item[1]))
    for ix, items in enumerate(output.items()):
        if ix >= top_ix:
            break
        key, val = items
        print(f"{ix}.", key, val)



def euclidean_numerical(num_feats : dict[str], input_track : Track, comparison_track : Track):
    """
    Essentia the Ln Norm function. For this, we have the following:
        * p = 1 : Manhattan Distance
        * p = 2 : Euclidean Distance

    By default, this will be set to Euclidean (p=2), however it must be noted that
    throughout testing, both Manhattan and Euclidean yielded amazing results. Other
    methods such as Mahalanobis and Cosine distance were not as good.

    NOTE : There are more distance methods yet to be tested such as Earth's Mover.
    """
    dist_score = 0
    for feature, weight in num_feats.items():
        input_feat   = input_track[feature]
        compare_feat = comparison_track[feature]
        dist_score   += ((input_feat - compare_feat)**2) * weight

    return float(dist_score)


# For now this is really only euclidean, don't care
def euclidean_dimensional(dim_feats : dict[str], input_track : Track, comparison_track : Track):
    dist_score = 0
    for feature, weight in dim_feats.items():
        input_feat = np.array(ast.literal_eval(input_track[feature]))
        comp_feat  = np.array(ast.literal_eval(comparison_track[feature]))

        dist_score += (np.linalg.norm(input_feat - comp_feat)) * weight

    return float(dist_score)


def cosine_dimensional(dim_feats : dict[str], input_track : Track, comparison_track : Track):
    dist_score = 0
    for feature, weight in dim_feats.items():
        input_feat = np.array(ast.literal_eval(input_track[feature]))
        comp_feat  = np.array(ast.literal_eval(comparison_track[feature]))

        # Avoid division by zero
        if np.linalg.norm(input_feat) == 0 or np.linalg.norm(comp_feat) == 0:
            cosine_distance   = 1.0  # Max distance if one of the vectors is zero
        else:
            cosine_similarity = np.dot(input_feat, comp_feat) / (np.linalg.norm(input_feat) * np.linalg.norm(comp_feat))
            cosine_distance   = 1 - cosine_similarity

        dist_score += cosine_distance * weight

    return float(dist_score)

### Testing
God

In [ ]:
# Also considering features from the previous tests
prev_features = ['popularity', 'mfcc_peak_energy', 'mfcc_avg_energy', 'pitch_salience', 'bpm', 'energy', 'integrated_loudness', 'loudness_range',
                'danceable_effnet', 'aggressive_effnet', 'happy_effnet', 'party_effnet', 'relaxed_effnet', 'sad_effnet', 'acoustic_effnet',
                'electronic_effnet', 'instrumental_effnet', 'female_effnet', 'tonal_effnet', 'bright_effnet', 'bright_nsynth_effnet',
                'approachable_effnet', 'approachability_effnet', 'engaging_effnet', 'engagement_effnet']


# Bring forward the new features for comparisons
curr_features  = ['mfcc_peak_energy', 'mfcc_avg_energy'	,'pitch_salience', 'avg_dissonance', 'avg_rollof', 'bpm', 'energy', 'integrated_loudness',
                  'loudness_range',	'danceable_effnet',	'aggressive_effnet', 'happy_effnet', 'party_effnet', 'relaxed_effnet', 'sad_effnet', 'acoustic_effnet',
                  'electronic_effnet',	'instrumental_effnet', 'female_effnet',	'tonal_effnet',	'bright_effnet','bright_nsynth_effnet',
                  'approachability_effnet',	'engagement_effnet', 'pitch_mean', 'pitch_var']


num_features  = [
    'popularity', 'mfcc_peak_energy', 'mfcc_avg_energy', 'pitch_salience', 'avg_dissonance', 'avg_rollof',
    'bpm', 'energy', 'integrated_loudness', 'loudness_range', 'danceable_effnet', 'aggressive_effnet',
    'happy_effnet', 'party_effnet', 'relaxed_effnet', 'sad_effnet', 'acoustic_effnet', 'electronic_effnet',
    'instrumental_effnet', 'female_effnet', 'tonal_effnet', 'bright_effnet', 'bright_nsynth_effnet',
    'approachability_effnet', 'engagement_effnet', 'pitch_mean', 'pitch_var', 'tristimulus' , 'pitch_hist'
]

dim_features = ['tristimulus', 'pitch_hist']


num_feats = {'acoustic_effnet': 1.0, 'aggressive_effnet': 1.0, 'approachability_effnet': 1.0,
            'avg_dissonance': 1.0, 'avg_rollof': 1.0, 'bpm': 1.0 , 'bright_effnet': 1.0,
              'bright_nsynth_effnet': 1.0, 'danceable_effnet': 1.0, 'electronic_effnet': 1.0,
                'energy': 1.0, 'engagement_effnet': 1.0, 'female_effnet': 1.0, 'happy_effnet': 1.0,
                  'instrumental_effnet': 1.0, 'integrated_loudness': 1.0, 'loudness_range': 1.0, 'mfcc_avg_energy': 1.0,
                    'mfcc_peak_energy': 1.0, 'party_effnet': 1.0,  'pitch_mean': 1.0, 'pitch_salience': 1.0, 'tonal_effnet': 1.0,
                      'pitch_var': 1.0, 'popularity': 1.0, 'relaxed_effnet': 1.0, 'sad_effnet': 1.0, }

dim_feats = {'pitch_hist': 0.9491799699087154, 'tristimulus': 0.8869536129879692}



num_w =  {'popularity': 0.3013510864425077, 'mfcc_peak_energy': 0.6977760416637019, 'mfcc_avg_energy': 2.404421446022957, 'pitch_salience': 1.1561506089354052, 'avg_dissonance': 1.5497180900481735, 'avg_rollof': 1.1300019734733604,
            'bpm': 2.1865291148890864, 'energy': 0.5027426356319643, 'integrated_loudness': 0.5166552666292368, 'loudness_range': 0.4011882701870904, 'danceable_effnet': 1.4076026007493962, 'aggressive_effnet': 0.7923839785622372,
              'happy_effnet': 1.0410429235019942, 'party_effnet': 0.6472759687689535, 'relaxed_effnet': 0.5851800399578627, 'sad_effnet': 0.9420298384780155, 'acoustic_effnet': 0.8599286165483203, 'electronic_effnet': 0.7758457620157805,
                'instrumental_effnet': 1.0154656212122504, 'female_effnet': 1.2307333536476226, 'tonal_effnet': 1.2685467432394164, 'bright_effnet': 1.0398004382566515, 'bright_nsynth_effnet': 1.3154840784632336, 'approachability_effnet': 1.0157499356608128,
                  'engagement_effnet': 0.7426274636394319, 'pitch_mean': 0.8464004668867382, 'pitch_var': 0.5772389660744255}

dim_w = {'tristimulus': 1.6018620847875267, 'pitch_hist': 1.9045036188521096}
dim_w = {'mfcc_mean': 1, 'mfcc_std': 1}


# for i in range(z_score_df.shape[0]):
input_track = z_score_df.iloc[3]

print(f"Selected Track : {input_track['artist']}  {input_track['title']}\n")
dist_helper(input_track = input_track,
            compare_df  = z_score_df,
            num_feats   = {},
            dim_feats   = dim_w,
            top_ix      = 21
            )
print('\n\n')




Selected Track : Volor Flex  Room 09

0. Volor Flex-Room 09 0.0
1. 454-REDRUM 0.004215472471958637
2. Madvillain-All Caps 0.004359182794815286
3. Burial-Homeless 0.004930711807217825
4. Cursive-The Radiator Hums 0.006050462139759238
5. Duster-Constellations 0.006064673637916518
6. Ecco2k-Cc 0.006085780417966924
7. Cocteau Twins-Hitherto 0.006157881832259182
8. Ecco2k-Peroxide 0.006299643464912075
9. Magdalena Bay-You Lose! 0.006689755756543514
10. Johnny Foreigner-Sometimes, in the Bullring 0.007177275045957221
11. Grimes-Pin 0.008387223588151804
12. bar italia-maddington 0.008464325673111284
13. Double Virgo-Chlorine 0.008537746063684515
14. Burial-Ghost Hardware 0.008665311927243846
15. Cocteau Twins-Musette and Drums 0.008814386036329225
16. Q and Not U-Nine Things Everybody Knows 0.008897309891776284
17. Bladee-LUCKY LUKE 0.009277451104838685
18. Burial-Archangel 0.010781764314673259
19. Ken Carson-ss 0.011297896763734161
20. Bladee-KING NOTHINGG 0.011313898193974437





: 

### Some Results

0. Frou Frou-Breathe In 0.0
1. Imogen Heap-Have You Got It In You? 4.2534212179851885
2. The Avalanches-Radio 7.132228705475651
3. Snow Strippers-Sad World 7.161188428890476
4. Chief Keef-Woulda Coulda 8.113625980126326
5. Playboi Carti-R.I.P. Fredo 8.3200176636156
6. Luis Fonsi-Regalame un Minuto Mas 8.509847857949348
7. Jeff Rosenstock-We Begged 2 Explode 9.713345464863831
8. Jai Paul-Crush 12.821721757028218
9. Beyonce-Protector 13.590239773270218
10. The Avalanches-Since I Left You 15.176086975739791
11. luan sounds-SANGRE 15.785023681714025
12. Shakira-Your Embrace 18.22773197682716
13. Evilgiane-DESIGNER DRUGZ 18.302148775447474
14. Animal Collective-My Girls 18.34102227282058
15. Mon Laferte-Tu Falta De Querer 18.466195118309795
16. the pillows-Runner High ~FLCL version~ 18.51421530273322
17. Pi'Erre Bourne-Joe Morris 19.620273989068153
18. Jhayco-Kobe En LA 2.0 19.7015174862037
19. Cities Aviv-REALMS 20.006551434363555